**# Step 1: Install Required Libraries**


In [ ]:
!pip install requests beautifulsoup4


# **Step 2: Fetch Data from the Web Links**
### *Now, a Python script to scrape the historical lottery data from the two links*

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Function to fetch Mega Millions data
def fetch_mega_millions_data():
    url = 'https://www.valottery.com/api/v1/downloadall?gameId=15'  # URL for Mega Millions data
    response = requests.get(url)
    content = response.content.decode('utf-8')  # Decode the byte response

    # Clean and process data
    data = []
    lines = content.splitlines()
    for line in lines:
        if re.match(r"\d{2}/\d{2}/\d{4};", line):  # Match date format
            parts = line.split(';')
            date = parts[0]
            numbers = parts[1].split(',')
            mega_ball = parts[2].split(':')[-1].strip()
            data.append([date] + numbers + [mega_ball])

    # Convert to DataFrame
    columns = ['Date', 'Number1', 'Number2', 'Number3', 'Number4', 'Number5', 'MegaBall']
    df = pd.DataFrame(data, columns=columns)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

    return df

# Function to fetch Powerball data
def fetch_powerball_data():
    url = 'https://www.valottery.com/api/v1/downloadall?gameId=20'  # URL for Powerball data
    response = requests.get(url)
    content = response.content.decode('utf-8')  # Decode the byte response

    # Clean and process data
    data = []
    lines = content.splitlines()
    for line in lines:
        if re.match(r"\d{2}/\d{2}/\d{4};", line):  # Match date format
            parts = line.split(';')
            date = parts[0]
            numbers = parts[1].split(',')
            powerball = parts[2].split(':')[-1].strip()
            data.append([date] + numbers + [powerball])

    # Convert to DataFrame
    columns = ['Date', 'Number1', 'Number2', 'Number3', 'Number4', 'Number5', 'Powerball']
    df = pd.DataFrame(data, columns=columns)
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

    return df

# Fetch data for Mega Millions and Powerball
mega_millions_data = fetch_mega_millions_data()
powerball_data = fetch_powerball_data()

# Display the fetched data
print("Mega Millions Data:")
print(mega_millions_data.head())

print("\nPowerball Data:")
print(powerball_data.head())


Mega Millions Data:
        Date Number1 Number2 Number3 Number4 Number5 MegaBall
0 2024-10-25      23      26      35      41      43        7
1 2024-10-22       8      43      48      58      60        4
2 2024-10-18       4       9      26      39      58       23
3 2024-10-15      22      34      44      54      62        3
4 2024-10-11       3      10      29      52      57       20

Powerball Data:
        Date Number1 Number2 Number3 Number4 Number5 Powerball
0 2024-10-26       8      12      40      45      51        15
1 2024-10-23       2      15      27      29      39        20
2 2024-10-21       1      25      57      62      64        15
3 2024-10-19       7      16      19      45      64        25
4 2024-10-16       4      30      39      44      60        11


# **Fetching Mega Millions and Powerball Data from APIs**
### *Requests to fetch the data and integrate it into our AI model.*
**Results for 6th Number may be over 25 and 26 Limit. Repurpose that number to inside the the line and replace it with a number under 25 or 26 to their respective games.**

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        # Check if the row has enough values to unpack
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)  # Limit split to 2
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest)
def random_forest_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network)
def neural_network_prediction(data):
    X = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data)

    # Neural Network
    nn_result = neural_network_prediction(data)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # MegaBall or Powerball prediction
    powerball_prediction = random.choice(data['MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'].values)

    print(f"Numbers: {final_prediction} | {'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'}: {powerball_prediction}")
    return final_prediction, powerball_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [17 20 46 16  8] | MegaBall: 23
Predicting for: MegaMillions
Numbers: [31  1 19  1 57] | MegaBall: 1
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [17  1 17 11 46] | MegaBall: 49
Predicting for: MegaMillions
Numbers: [46 20 10 31 20] | MegaBall: 23
Predicting for: MegaMillions
Numbers: [23 17 46 40  9] | MegaBall: 21

Powerball Predictions:
Predicting for: Powerball
Numbers: [32 39 32  4 34] | Powerball: 23
Predicting for: Powerball
Numbers: [23 21 36 19  1] | Powerball: 24
Predicting for: Powerball
Numbers: [36  5  7 23  1] | Powerball: 14
Predicting for: Powerball
Numbers: [ 1 18 17 35 23] | Powerball: 21
Predicting for: Powerball
Numbers: [ 4 36 15 21 23] | Powerball: 23


# **Calculate Deciles:**
Compute deciles for each set of drawn numbers (i.e., for each row) and append those decile values as new features.
# **Modify Models to Use Deciles:**
Both the random forest and neural network models will be updated to consider the deciles as additional features.

In [3]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        # Check if the row has enough values to unpack
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)  # Limit split to 2
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)

    # Add decile calculations
    df = add_deciles(df)

    return df

# Add decile columns to the dataset
def add_deciles(df):
    for col in ['Number1', 'Number2', 'Number3', 'Number4', 'Number5']:
        df[col + '_Decile'] = pd.qcut(df[col], 10, labels=False)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest) with deciles
def random_forest_prediction(data):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5',
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile']

    X = data[features].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network) with deciles
def neural_network_prediction(data):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5',
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile']

    X = data[features].values
    y = data[['MegaBall' if 'MegaBall' in data.columns else 'Powerball']].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data)

    # Neural Network
    nn_result = neural_network_prediction(data)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # MegaBall or Powerball prediction
    powerball_prediction = random.choice(data['MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'].values)

    print(f"Numbers: {final_prediction} | {'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'}: {powerball_prediction}")
    return final_prediction, powerball_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [ 2 10 13 20  1] | MegaBall: 13
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [10 35 15 31  6] | MegaBall: 38
Predicting for: MegaMillions
Numbers: [19  8 31 42 17] | MegaBall: 52
Predicting for: MegaMillions
Numbers: [17  9 35 31  3] | MegaBall: 20
Predicting for: MegaMillions


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [27 17  1 10  1] | MegaBall: 22

Powerball Predictions:
Predicting for: Powerball
Numbers: [21 36 23 23 23] | Powerball: 30
Predicting for: Powerball
Numbers: [ 1 32 33 23 21] | Powerball: 5
Predicting for: Powerball
Numbers: [15 39 23 32 21] | Powerball: 22
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [39 21 36 44  4] | Powerball: 15
Predicting for: Powerball
Numbers: [ 1  5 36 32 24] | Powerball: 24


# **Extract Deciles for the Sixth Number:**
Add a decile column for the sixth number in each game.
# **Modify the Prediction Models:**
 Adjust the random forest and neural network models to also predict the sixth number based on the first five numbers and the sixth number's decile.

In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# API URLs for Mega Millions and Powerball
MEGA_MILLIONS_API = 'https://www.valottery.com/api/v1/downloadall?gameId=15'
POWERBALL_API = 'https://www.valottery.com/api/v1/downloadall?gameId=20'

# Fetch Mega Millions Data
def fetch_mega_millions_data():
    response = requests.get(MEGA_MILLIONS_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'MegaBall')
    else:
        raise Exception("Failed to fetch Mega Millions data")

# Fetch Powerball Data
def fetch_powerball_data():
    response = requests.get(POWERBALL_API)
    if response.status_code == 200:
        return process_api_data(response.text, 'Powerball')
    else:
        raise Exception("Failed to fetch Powerball data")

# Process the API data (assuming CSV format with semicolons)
def process_api_data(data, ball_type):
    # Load the data into a pandas dataframe
    rows = data.splitlines()
    structured_data = []
    for row in rows:
        if row.count(';') >= 2:  # Check for at least 2 semicolons
            date, numbers, extra_ball = row.split(';', 2)
            num_list = [int(n) for n in numbers.split(',')]
            ball = int(extra_ball.split(':')[-1])
            structured_data.append(num_list + [ball])

    columns = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]
    df = pd.DataFrame(structured_data, columns=columns)

    # Add decile calculations
    df = add_deciles(df, ball_type)

    return df

# Add decile columns to the dataset, including the sixth number
def add_deciles(df, ball_type):
    for col in ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type]:
        df[col + '_Decile'] = pd.qcut(df[col], 10, labels=False)
    return df

# Frequency Analysis
def frequency_analysis(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    freq = np.bincount(numbers)
    return np.argsort(freq)[-5:]  # Return 5 most frequent numbers

# Supervised Learning (Random Forest) with deciles
def random_forest_prediction(data, ball_type):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type,
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile', f'{ball_type}_Decile']

    X = data[features].values
    y = data[ball_type].values.ravel()

    model = RandomForestClassifier(n_estimators=100)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Deep Learning (Neural Network) with deciles
def neural_network_prediction(data, ball_type):
    features = ['Number1', 'Number2', 'Number3', 'Number4', 'Number5', ball_type,
                'Number1_Decile', 'Number2_Decile', 'Number3_Decile', 'Number4_Decile', 'Number5_Decile', f'{ball_type}_Decile']

    X = data[features].values
    y = data[ball_type].values.ravel()

    model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=500)
    model.fit(X, y)

    predictions = model.predict(X[-5:])
    return predictions

# Genetic Algorithm for evolving predictions
def genetic_algorithm_optimization(predictions, data):
    population_size = 100
    generations = 50
    population = np.random.randint(1, 70, size=(population_size, 5))  # 1-70 for the numbers

    def fitness(individual):
        return np.sum(np.isin(individual, predictions))  # Fitness based on match with predictions

    for _ in range(generations):
        fitness_scores = np.array([fitness(ind) for ind in population])
        selected = population[np.argsort(fitness_scores)[-20:]]  # Select top 20
        next_generation = []
        for _ in range(population_size):
            parent1, parent2 = selected[np.random.choice(len(selected), 2, replace=False)]
            crossover = np.random.randint(1, 5)
            child = np.concatenate([parent1[:crossover], parent2[crossover:]])
            next_generation.append(child)
        population = np.array(next_generation)

    best_individual = population[np.argmax(fitness_scores)]
    return best_individual

# MCMC for random probabilistic prediction
def mcmc_prediction(data):
    numbers = data[['Number1', 'Number2', 'Number3', 'Number4', 'Number5']].values.flatten()
    probs = np.bincount(numbers) / len(numbers)
    return np.random.choice(range(len(probs)), p=probs, size=5, replace=False)

# Combine all techniques to get predictions, including the sixth number
def combined_predictions(data, lottery_type='MegaMillions'):
    print(f"Predicting for: {lottery_type}")

    ball_type = 'MegaBall' if lottery_type == 'MegaMillions' else 'Powerball'

    # Frequency Analysis
    freq_analysis_result = frequency_analysis(data)

    # Random Forest
    rf_result = random_forest_prediction(data, ball_type)

    # Neural Network
    nn_result = neural_network_prediction(data, ball_type)

    # Genetic Algorithm Optimization
    ga_result = genetic_algorithm_optimization(freq_analysis_result, data)

    # MCMC Result
    mcmc_result = mcmc_prediction(data)

    # Combine predictions (ensemble approach)
    combined_results = np.concatenate([freq_analysis_result, rf_result, nn_result, ga_result, mcmc_result])
    final_prediction = np.random.choice(combined_results, size=5, replace=False)  # Choose 5 final numbers

    # Predict the sixth number using the neural network model
    sixth_number_prediction = random.choice(nn_result)

    print(f"Numbers: {final_prediction} | {ball_type}: {sixth_number_prediction}")
    return final_prediction, sixth_number_prediction

# Main function to predict 5 results for MegaMillions and Powerball
def main():
    # Fetch historical data
    mega_millions_data = fetch_mega_millions_data()
    powerball_data = fetch_powerball_data()

    # Generate 5 predictions for Mega Millions
    print("Mega Millions Predictions:")
    for _ in range(5):
        combined_predictions(mega_millions_data, 'MegaMillions')

    # Generate 5 predictions for Powerball
    print("\nPowerball Predictions:")
    for _ in range(5):
        combined_predictions(powerball_data, 'Powerball')

# Run the prediction script
main()


Mega Millions Predictions:
Predicting for: MegaMillions
Numbers: [ 5 17  8 46 17] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [ 8 46 24 46  6] | MegaBall: 17
Predicting for: MegaMillions
Numbers: [17  1 14 46 46] | MegaBall: 8
Predicting for: MegaMillions
Numbers: [17 19 31 46 54] | MegaBall: 8
Predicting for: MegaMillions
Numbers: [20 10  1  4 46] | MegaBall: 8

Powerball Predictions:
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [ 1 32 36 39 23] | Powerball: 15
Predicting for: Powerball
Numbers: [39 37 31 21  4] | Powerball: 4
Predicting for: Powerball
Numbers: [31 41 20 23 32] | Powerball: 1
Predicting for: Powerball
Numbers: [39 28 34 15 54] | Powerball: 4
Predicting for: Powerball


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Numbers: [39  1  4 23 32] | Powerball: 4
